In [ ]:
import sys
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

#%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
if not tf.config.list_physical_devices('GPU'):
    print("감지된 GPU가 없습니다. GPU가 없으면 LSTM과 CNN이 매우 느릴 수 있습니다.")

# RNN과 어텐션을 사용한 NLP(자연어 처리)

### Char-RNN을 사용해 셰익스피어 문체 생성

#### 훈련 데이터셋 만들기

In [ ]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt" # 단축 URL
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

그다음 모든 글자를 정수로 인코딩해야함. 13장처럼 사용자 정의 전처리 층을 만드는 것이 한 방법.

텍스트에서 사용되는 모든 글자를 찾아 각기 다른 글자 ID에 매핑.

이 ID는 1부터 시작해 고유한 글자수까지 만들어짐(나중에 보겠지만, 마스킹에 사용하기 때문에 0부터 시작 X).

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

char_level=True로 지정하여 단어 수준 인코딩 대신 글자 수준 인코딩을 만듦.

이 클래스는 기본적으로 텍스트를 소문자로 바꿈(원치 않을경우 lower=False로 지정).

이제 문장을 글자 ID로 인코딩하거나 반대로 디코딩 가능 -> 텍스트의 고유 글자 수, 전체 글자 수 알 수 있음.

In [ ]:
tokenizer.texts_to_sequences(["First"])

In [ ]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

In [ ]:
max_id = len(tokenizer.word_index) # 고유한 문자 개수
dataset_size = tokenizer.document_count # 전체 문자 개수

In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

텍스트에 있는 글자를 섞으면 안 됨.

#### 순차 데이터셋을 나누는 방법

시계열을 다룰 때는 보통 시간에 따라 나눔. 

암묵적으로 RNN이 과거에서 학습하는 패턴이 미래에도 등장한다고 가정.

다른말로 이 시계열 데이터가 변하지 않는다고 가정.

많은 경우 시계열에서 이 가정이 타당. 하지만 다른 많은 시계열은 그렇지 않음. -> 어쩌라는거임?

텍스트의 처음 90%를 훈련 세트로 사용(나머지는 검증+테스트로 사용).

이 세트에서 한 번에 한 글자씩 반환하는 tf.data.Dataset 객체를 만듦.

In [ ]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

#### 순차 데이터를 윈도 여러 개로 자르기

훈련세트는 백만개 이상의 글자로 이뤄진 시퀀스 하나. 여기에 신경망 직접 훈련 X.

이 RNN은 백반 개의 층이 있는 심층 신경망과 비슷하고 매우 긴 샘플 하나로 훈련하는 셈.

대신 window()를 사용해 긴 시퀀스를 작은 텍스트 윈도로 변환시킬 수 있음.

In [ ]:
n_steps = 100
window_length = n_steps + 1 # 타깃 = 한 글자 앞선 입력
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

##### TIP_

n_steps을 튜닝가능. 짧은 입력 시퀀스에서 RNN을 훈련하는 것은 쉽지만 당연히 이 RNN은

n_steps보다 긴 패턴 학습 불가능. 따라서 너무 짧게 만들어선 안 됨.

기본적으로 window()는 원도를 중복하지 않음. 

shift=1 : 로 지정하면 가장 큰 훈련세트 만들수 있음.

첫번째 윈도 : 0~100번째 글자 포함, 두번째 윈도 : 1~101번쨰 글자 포함, ... (패딩없이 배치 데이터 만들기 위해)

drop_remainder=True : 모든 윈도가 동일하게 101개의 글자를 포함하도록 함 (그렇지 않으면 점점 줄어듦)

window()는 각각 하나의 데이터셋으로 표현되는 윈도를 포함하는 데이터셋을 만듦. -> 중첩 데이터셋

하지만 모델이 바로 사용할 순 없음. 따라서 중첩 데이터셋을 플랫 데이터셋으로 변환하는 flat_map() 호출.

{{1,2}, {3,4,5}} -> {1,2,3,4,5} : flat_map() 역할

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

윈도마다 batch(window_length)를 호출. 이 길이는 윈도 길이와 같기 때문에 텐서 하나를 담은 데이터셋을 얻음.

-> 이 데이터셋은 연속된 101 글자 길이의 윈도를 담음. 

경사하강법(4장)은 훈련세트 샘플이 동일 독립 분포일 때 잘 작동 -> 윈도를 섞어야 함.

윈도를 배치로 만들고 입력(처음 100개 글자)과 타깃(마지막 100개 글자)을 분리하는 코드

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

13장에서 처럼 일반적으로 범주형 입력 특성은 원-핫 벡터나 임베딩으로 인코딩되야함.

여기선 고유 글자 수가 적기 때문에(39개) 원-핫 벡터를 사용해 인코딩.

In [ ]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

마지막으로 프리패칭 추가

In [ ]:
dataset = dataset.prefetch(1)

In [ ]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

#### Char-RNN 모델 만들고 훈련

**노트**: `GRU` 클래스는 다음 매개변수에서 기본값을 사용할 때에만 GPU를 사용합니다: `activation`, `recurrent_activation`, `recurrent_dropout`, `unroll`, `use_bias` `reset_after`. 이 때문에 (책과는 달리) `recurrent_dropout=0.2`를 주석 처리했습니다.

이전 글자 100개를 기반으로 다음 글자를 예측하기 위해 유닛 128개를 가진 GRU층 2개와

입력(dropout)과 은닉 상태(recurrent_dropout)에 20% 드롭아웃. 필요하면 하이퍼파라미터 수정.

출력층은 15장에서 본 TimeDistributed 클래스를 적용한 Dense 층.

텍스트에 있는 고유한 글자 수 : 39개 -> 이 층의 유닛(max_id) : 39개

타임 스텝에서 출력 확률의 합은 1이어야 하므로 Dense층의 출력에 소프트맥스 함수 적용.

손실 : sparse_categorical_crossentropy, 옵티마이저 : adam

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

#### Char-RNN 모델 사용하기

모델에 새로운 텍스트를 주입하려면 앞에서와 같이 먼저 전처리를 해야함.

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

이제 이 모델을 사용해 어떤 텍스트의 다음 글자를 예측해봅시다.

In [ ]:
X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

#### 가짜 셰익스피어 텍스트 생성

tf.random.categorical() 함수를 사용해 모델이 추정한 확률을 기반으로 다음 글자를 무작위로 선택가능.

categorical()는 클래스의 로그 확률을 전달하면 랜덤하게 클래스 인덱스 샘플링.

생성된 텍스트의 다양성을 더 많이 제어하려면 온도라고 불리는 숫자로 로짓을 나눔.

온도가 0에 가까울수록 높은 확률을 가진 글자를 선택. 온도가 매우 높으면 모든 글자가 동일한 확률을 가짐.

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

next_char()를 반복 호출해 다음 글자를 얻고 텍스트에 추가하는 작은 함수를 만듦.

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))

In [ ]:
print(complete_text("t", temperature=1))

In [ ]:
print(complete_text("t", temperature=2))

더 좋은 텍스트를 생성하려면 GRU 층과 층의 뉴런 수를 늘리고 더 오래 훈련하거나 규제 추가

(ex. dropout 조절). 또한 이 모델은 현재 글자 100개인 n_steps보다 긴 패턴을 학습할 수 없음.

윈도를 크게할 순 있지만 훈련이 어려워짐. LSTM과 GRU 셀이라도 매우 긴 시퀀스는 다룰 수 없음.

#### 상태가 있는 RNN

RNN이 한 훈련 배치를 처리한 후에 마지막 상태를 다음 훈련배치의 초기상태로 사용하면 어떨까?

이렇게 하면 역전파는 짧은 시퀀스에서 일어나지만 모델이 장기간 패턴을 학습가능.

먼저 상태가 있는 RNN은 배치가 있는 각 입력 시퀀스가 이전 배치의 시퀀스가 끝난 지점에 시작.

따러서 할 일 들:

1. 할 일은 순차적이고 겹치지 않는 입력 시퀀스를 만드는 것(상태가 없는 RNN을 훈련하기 위해 사용한 섞은 뒤 겹쳐진 시퀀스가 아님.)

Dataset을 만들 때 window()에서 (shift=1 대신에) shift=n_steps를 사용.

또한 shuffle()를 호출해선 안 됨. -> 힘듦.

실제 batch(32)라고 호출하면 32개의 연속적인 윈도가 같은 배치에 들어감.

이 윈도가 끝난 지점부터 다음 배치가 계속되지 않음. 

첫번째 배치는 윈도 1~32까지 포함. 두번째 배치는 윈도 33~64까지 포함.

따라서 각 배치의 첫번째 윈도를 생각하면(즉, 윈도 1과 33) 연속적이지 않음.

-> 간단한 해결책 : 하나의 윈도를 갖는 배치를 만듦.

In [ ]:
'''
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)
'''

텍스트를 길이가 동일한 32개의 텍스트로 나누고 각 텍스트에 대해 연속적인 입력 시퀀스를 가진 데이터셋

하나를 만들 수 있음. 마지막으로 tf.data.Dataset.zip(datasets).map(lambda *windows: tf.stack(windows))

를 사용해 연속적인 배치를 만듦. 여기서 한 배치에서 n번째 입력 시퀀스의 시작은 정확히 이전 배치의

n번째 입력 시퀀스의 시작은 정확히 이전 배치 n번째 끝나는 지점.

In [ ]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

이제 상태가 있는 RNN을 만들어봅시다. 

첫째, 각 순환 층을 만들 때 stateful=True로 지정.

둘째, 상태가 있는 RNN은 배치 크기를 알아야 함(배치에 있는 입력 시퀀스의 상태를 보존해야 하기 때문).

따라서 첫번째 층에 batch_input_shape 매개변수를 지정해야 함. 

입력은 어떤 길이도 가질 수 있으므로 2번째 차원은 지정하지 않아도 됨.

**노트**: 여기에서도 GPU 가속을 위해 (책과 달리) `recurrent_dropout=0.2`을 주석 처리합니다.

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

에포크 끝마다 텍스트를 다시 시작하기 전에 상태를 재설정해야 함. callback 사용

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

지금까지 글자 수준 모델을 만들었습니다. 이제 단어 수준 모델을 살펴보고 자주 등장하는 자연어 처리 작업인

감성 분석을 다루어봅시다. (마스킹도 사용해 길이가 다른 시퀀스 다루는 방법도 알아봄.)

### 감성 분석

MNIST가 컴퓨터 비전계의 'hello world', IMDb 리뷰 데이터셋은 자연어 처리계 'hello world'.

In [ ]:
tf.random.set_seed(42)
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]

X_train은 각 리뷰들의 리스트. 각 리뷰는 정수 배열로 표현되며 각 정수는 하나의 단어를 나타냄.

구두점을 모두 제거하고 단어는 소문자로 변환한 다음 공백으로 나누어 빈도에 따라 인덱스를 붙임.

정수는 0,1,2 특별. 각각 패딩 토큰, SOS 토큰, 알 수 없는 단어를 의미. 다음과 같이 디코딩가능

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

실전 프로젝트에선 직접 텍스트를 전처리 해야함. Tokenizer 클래스 사용가능.

이번엔 (기본값인) char_level=False로 설정. 

단어를 인코딩할 때는 구두점, 줄바꿈, 탭을 포함해 많은 글자가 제외됨(filters로 바꿀 수 있음).

하지만 모든 문자가 단어 사이 공백을 사용하지는 않음. 

San Francisco나 #ILoveDeepLearning와 같은 경우를 생각해봅시다.

In [ ]:
datasets.keys()

다행히 더 좋은 방법이 있음. 단어수준으로 텍스트를 토큰화하거나 복원하는 비지도 학습 방법이 있음.

이 방법은 공백을 하나의 문자로 취급하기 때문에 언어 독립적. 

-> 모델이 이전에 본 적 없는 단어를 만나도 의미 추측가능.

::: 구글의 센텐스피스 프로젝트는 다쿠 구도와 존 리처드슨이 쓴 논문의 오프 소스가 있음.

다른 방법으로 부분 단어를 인코딩하는 방법이 있음(바이트 페어 인코딩).

마지막으로 종요한 하나는 텐서플로 팀이 2019년에 릴리스한 TF.Text 라이브러리

(바이트 페어 인코딩의 변종인) 워드피스를 포함해 다양한 토큰화 전략이 구현되어 있음.

먼저 (13장에서 소개한) 텐서플로 데이터셋을 사용해 원본 IMDb 리뷰를 텍스트(바이트 스트링)으로 적재.

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [ ]:
train_size, test_size

In [ ]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

In [ ]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [ ]:
preprocess(X_batch, y_batch)

리뷰 텍스트를 잘라내어 각 리뷰에서 처음 300자만 남김. 이렇게 하면 훈련속도를 높일 수 있음.

또 일반적으로 처음 한두문장에서 리뷰가 긍정적인지 아닌지 판단가능하기에 성능에 영향을 미치지 않음.

그다음 정규식을 사용해 <br /> 태그를 공백으로 바꿈.

문자와 작은 따옴표가 아닌 다른 모든 문자를 공백으로 바꿈.

마지막으로 preprocess()는 리뷰를 공백으로 나눔. 이떄 래그드 텐서가 반환됨.

이 래그드 텐서를 밀집 텐서로 바꾸고 동일한 길이가 되도록 패딩 토큰 "<pad>"로 모든 리뷰 패딩

그다음 어휘 사전을 구축해야함. 전체 훈련 세트를 한 번 순회하면서 preprocess()를 적용해 

Counter로 단어 등장 횟수를 셈.

In [ ]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

가장 많이 등장하는 단어 3개를 확인

In [ ]:
vocabulary.most_common()[:3]

아마 좋은 성능을 내기 위해 사전에 있는 모든 단어를 모델이 알아야 할 필요는 없을 것.

어학사전 중 가장 많이 등장하는 단언 1만개만 남기고 삭제.

In [ ]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [ ]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

이제 각 단어를 ID(즉, 어휘 사전의 인덱스)로 바꾸는 전처리 단계를 추가함. 13장에서 한 것처럼

1,000개의 oov 버킷을 사용하는 룩업 테이블을 만듦.

In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

이 테이블에서 단어 몇 개에 대한 ID를 확인해보겠습니다.

In [ ]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

단어 this, movie, was는 룩업 테이블에 있으므로 이 단어들의 ID는 10,000보다 적음.

단어 faaaaaantastic은 없기 때문에 10,000보다 크거나 같은 ID를 가진 oov버킷 중 하나에 매핑.

##### TIP : TF 변환(13장 참조)에서 어휘 사전을 편리하게 다룰 수 있는 함수 제공

이제 최종훈련세트를 만들 준비됨. 리뷰를 배치로 묶고 preprocess()를 사용해 단어의 짧은 시퀀스로 바꿈.

그다음 앞서 만든 테이블을 사용하는 encode_words()로 단어를 인코딩함. 마지막으로 다음 배치를 프리패치

In [ ]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

모델 만들어서 훈련 :

In [ ]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

첫 번째 층은 단어 ID를 임베딩으로 변환하는 Embedding층 (13장 참조).

임배딩 행렬은 단어 ID당(vocab_size + num_oov_buckets) 하나의 행과 임베딩 차원당

(이 예에선 128차원을 사용하나 이는 튜닝가능한 하이퍼파라미터) 하나의 열을 가짐.

모델의 입력은 [배치크기, 타임 스텝 수] 크기를 가진 2D 텐서이지만 

Embedding 층의 출력은 [배치크기, 타임 스텝 수, 임베딩 크기] 크기를 가진 3D 텐서가 됨.

|

GRU 층 2개로 구성되고 2번째 층은 마지막 타임스텝의 출력만 반환함.

출력층은 시그모이드 활성화 함수를 사용하는 하나의 뉴런. 리뷰가 영화에 대한 긍정적인 감정을

표현하는지에 대한 추정확률을 출력함.

#### 마스킹

In [ ]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

구체적으로 마스크 텐서(입력과 크기가 같은 bool 텐서)를 만듦.

ID가 0인 위치는 False이고 나머지는 True. 이 마스크 텐서는 모델에 의해 이어지는 모든 층에 

타임스텝 차원이 유지되는 한 자동으로 전파됨.

이 예에선 2개의 GRU층이 자동으로 이 마스크 텐서를 받음. 2번째 GRU층이 시퀀스를 반환하지 않기 때문에

(이 층은 마지막 타임스텝의 출력만 반환). 순환 층은 마스킹된 타임스텝을 만나면 이전 타임스텝의 출력을 

단순히 복사. 만약 마스크가 출력에도 전파된다면 손실에도 적용가능(이 예제는 해당되지 않음).

따라서 마스킹된 타임 스텝은 손실에 영향을 미치지 못할 것(이 타임스텝 손실이 0).

#### 사전 훈련된 임베딩 사용하기

고정 디렉토리 다운로드 :

In [ ]:
tf.random.set_seed(42)
TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [ ]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

**문장 인코더** : 문자열을 입력으로 받아 하나의 벡터로 인코딩(이 예제는 50차원 벡터).

내부적으로는 문자열을 파싱(공백으로 단어 나눔)햐소 대규모 코퍼스에서 사전훈련된 임베딩 행렬을 사용해

각 단어를 임베딩. 이 코퍼스는 구글 코퍼스 7B 코퍼스(전체 단어수 70억개).

그다음 모든 단어임베딩의 평균을 계산. 이 결과가 문장 임베딩.

그다음 2개의 Dense 층을 추가해 감성 분석 모델을 만듦. 기본적으로 hub.KerasLayer 층은 훈련되지 않음.

하지만 이 층을 만들 때 trainable=True로 설정해 작업에 맞게 미세 조정가능.

그다음 IMDb 리뷰 데이터셋을 다운로드. (배치와 프리페치를 제외하고) 따로 전처리 할 필요가 없음.

바로 모델을 훈련할 수 있음.

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

### 신경망 기계 번역을 위한 인코더-디코더 네트워크

* 지금까지는 모든 (incoder&decoder) 입력 시퀀스의 길이가 동일하다고 가정.

  당연히 문장의 길이는 다름. 일반적인 텐서는 크기가 고정되어 있기 때문에 동일한 길이의 문장만 담을 수 있음.

  앞서 설명한 것처럼 마스킹을 사용해 처리가능. 허나 문장길이가 많이 다르면 감성분석에서 한 것처럼

  그냥 잘라낼 수 없음(잘려진 번역이 아닌 전체문장을 번역해야 하기에).

  대신 문장을 비슷한 길이의 버킷으로 그룹핑(ex, 한 버킷은 1~6개 단어로 이뤄진 문장을 담고

  또 다른 버킷은 7~12개 단어로 이뤄진 문장을 담는 식).

  버킷에 담긴 문장이 모두 동일한 길이가 되도록 패딩을 추가.

  ex) 'I drink milk'는 '<pad> <pad> <pad> milk drink I'가 됨.

* EOS 토큰 이후 출력은 모두 무시. 이 토큰들은 손실에 영향을 미치지 않음(마스킹 처러되야함). 

  예를 들어 모델이 'Je bois du lait <eos> oui'를 출력하면 마지막 단어에 대한 손실은 무시.

* 출력 어휘 사전이 (이 예제처럼) 방대할 경우 모든 단어의 확률을 출력하려면 매우 느려짐.

  예를 들어 어휘 사전이 프랑스어 단어 5만개를 가진다면 디코더는 5만차원의 벡터를 출력할 것.

  이런 큰 벡터에서 소프트맥스 함수를 계산하는 것은 연산비용이 매우 높음. 이를 피하기 위한 한가지 방법은

  타깃단어에 대한 로짓과 타깃이 아닌 단어 중 무작위로 샘플링한 단어의 로짓만 고려하는 것.

  훈련 시에 tf.nn.sampled_softmax_loss()를 사용하고 추론 시에는 일반 소프트맥스 함수를 사용가능.

  (샘플링 소프트맥스는 타깃을 알고 있어야 하므로 추론 시에는 사용가능)

In [ ]:
tf.random.set_seed(42)
vocab_size = 100
embed_size = 10

기본적인 인코더-디코더 모델 :

In [ ]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

이 코드는 대부분 이해 되지만 몇 가지 언급이 필요함.

먼저 LSTM층을 만들 때 최종 은닉 상태를 디코더로 보내기 위해 return_state=True로 지정.

LSTM 셀을 사용하기 때문에 은닉 상태 2개(장기와 단기)를 반환.

TrainingSampler는 텐서플로 애드온에 포함되어 있는 여러 샘플러 중 하나.

이 샘플러는 각 스텝에서 디코더에게 이전스텝의 출력이 무엇인지 알려줌.

추론 시에는 실제로 출력되는 토큰의 임베딩이 됨. 훈련시에는 이전 타깃 토큰의 임베딩이 되야함.

때문에 TrainingSampler를 사용. 실전에서는 이전 타임스텝의 타깃의 임베딩을 사용해 훈련을 시작해 이전 스텝에서

출력된 실제 토큰의 임베딩으로 점차 바꾸는 것이 좋음.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

#### 양방향 RNN

각 타임스텝에서 일반적인 순환 층은 과거와 현재의 입력만 보고 출력을 생성.

다른말로 인과적. 즉 미래를 볼 수 없다는 뜻. 이런 종류의 RNN은 시계열을 예측할 때 적합하지만 

신경망 기계 번역 같은 여러 종류 NLP 작업에는 맞지 않음. 이런 작업은 주어진 단어를 인코딩하기 전에

다음 단어를 미리 보는 것이 좋음. 이를 위해 동일한 입력에 대해 2개의 순환 층을 실행함.

하나는 왼쪽에서 오른쪽으로 단어를 읽고 다른 하나는 오른쪽에서 왼쪽으로 읽음.

그다음 일반적으로 타임 스텝마다 이 두 출력을 연결 -> 양방향 순환 층

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
])

model.summary()

##### NOTE_ 

Bidirectional층은 GRU 층을 복사함(반대 방향으로), 그다음 두 층을 실행해 그 출력을 연결.

GRU 층이 10개의 유닛을 가지면 Bidirectional층은 타임 스텝마다 20개의 값을 출력함.

#### 빔 검색

모델이 앞선 실수를 고칠 수 있게 할까? -> 빔 검색 (방법 중 하나)

k개의 가능성 있는 (예를 들면 상위 3개) 문장의 리스트를 유지하고 디코더 단계마다 이 문장의 단어를 하나씩

생성해 가능성있는 k개의 문장을 만듦. 파라미터를 k를 빔 너비라고 부름.

1. "start" 토큰이 입력된다.

2. "start" 입력을 바탕으로 나온 예측 값의 확률 분포 중 가장 높은 확률 K개를 고른다.

  (이제부터 이 K개의 갈래는 각각 하나의 빔이 됩니다)

3. K개의 빔에서 각각 다음 예측 값의 확률 분포 중 가장 높은 K개를 고른다.

  ( 이를 자식 노드라고 하겠습니다 )

4. 총 K2개의 자식 노드 중 누적 확률 순으로 상위 K개를 뽑는다.

  ※ 빔서치에서 고려하는 모든 확률은 누적 확률입니다. 어떠한 자식 노드들이 서로 같은 확률을 가지더라도, 
  
  어떤 빔에서 뻗어나왔냐에 따라 누적 확률은 달라지게 됨

5. 뽑힌 상위 K개의 자식노드를 새로운 빔으로, 다시 상위 K개의 자식 노드를 만든다.

6. "eos"를 만난 빔이 K개가 될 때까지 Step3 - Step4를 반복.

예를 들어 빔 너비 3의 빔 검색으로 'Comment vas-tu?'를 번역한다고 가정.

1번째 디코더 스텝에서 모델이 가능한 모든 단어에 대한 추정확률을 출력할 것.

최상위 3개 단어의 추정확률을 How(75%), What(3%), You(1%)라고 가정. -> 현재 리스트

|

그다음 3개의 모델로 복사해 각 문장의 다음 단어를 찾음. 

각 모델은 어휘사전에 있는 단어에 해당하는 추정확률 출력.

1번째 모델은 How 문장에 이어질 단어찾음. 단어 will에 대해 36%, are에 대해 32%, do에 대해 16%의 확률 출력.

실제로 이 값은 How로 시작하는 문장이 주어졌을 때의 조건부 확률. 2번째 모델은 What 문장을 이어나감.

단어 are에 대해 50%의 조건부 확률을 출력하는 식.

어휘사전 1만개가 있다고 가정시 각 모델은 1만개의 확률 추정.

그다음 이 모델이 2개의 단어로 이뤄진 3만개의 문장에 대해 확률을 계산.

이를 위해 완성된 문장된 문장의 추정된 확률에 각 단어의 추정된 조건부 확률을 곱함.

예를 들어 문장 How의 추정확률이 75%이고 (1번째 단어로 How가 주어졌을 때)

단어 will에 대한 추정된 조건부 확률이 36%라면 "How will" 문장의 추정 확률은 75%*36%=27%

두 단어로 이뤄진 문장 3만개 확률을 계산한 후 최상위 3만 추림.

이들은 모두 How로 시작, 예를 들어 'How will'(27%), 'How are'(24%), 'How do'(12%)

|

그다음 동일한 과정 반복. 모델 셋을 사용해 3문장에서 다음 단어 예측.

그다음 3단어로 이뤄진 문장 3만개 확률을 계산. 가령 가장 높은 확률을 가진 3문장은

'How are you'(10%), 'How do you'(8%), 'How will you'(2%).

다음 단계에서 'How do you do'(7%), 'How are you [EOS]'(6%), 'How are you doing'(3%).

여기서 'How will'이 제외되었고 남은 번역 3개는 모두 납득할 만함.

In [ ]:
beam_width = 10
decoder = tfa.seq2

### 어텐션 메커니즘

예를 들어 디코더가 단어 lait를 출력해야 하는 타임 스텝에서 단어 milk에 주의를 집중.

입력단어에서 번역까지 경로가 훨씬 짧아지는 것을 의미하므로 

RNN의 단기 기억의 제한성에 훨씬 적은 영향을 받게됨.

어텐션 매커니즘은 신경망 기계번역(그리고 일반적인 NLP)에 큰 변화를 만듦.

특히(30단어 이상의) 긴 문장에 대해서 최고 수준의 성능을 크게 향상시킴.

#### 비주얼 어텐션

어텐션 메커니즘은 이제 다양한 목적으로 사용됨. NMT 이외의 목적으로 사용된 첫번째 애플리케이션은

비주얼 어텐션을 사용한 이미지 캡션 생성.

|

합성곱 신경망이 먼저 이미지를 처리해 일련의 특성 맵을 출력.

그다음 어텐션 메커니즘을 장착한 디코더 RNN이 한 번에 한 단어씩 캡션 생성.

디코더 타임스텝마다(단어마다) 디코더는 어텐션 모델을 사용해 이미지에서 적절한 부위에 초점을 맞춤.

#### 트랜스포머 구조: 어텐션이 필요한 전부다

이 구조는 순환 층이나 합성곱 층을 전혀 사용하지 않고 어텐션 메커니즘만(그리고 임베딩 층, 밀집 층, 정규화 층,

몇 가지 다른 구성 요소를 더해) 사용해 NMT 문제에서 최고 수준 성능을 크게 향상.

추가적인 장점은 이 구조를 훨씬 빠르게 훈련가능 + 병렬화 쉬움.

따라서 이전 모델에서 최고 성능을 내기 위해 필요한 시간과 비용 일부만으로 훈련가능

##### **위치 인코딩**

In [ ]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1: max_dims += 1 # max_dims must be even
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

In [ ]:
max_steps = 201
max_dims = 512
pos_emb = PositionalEncoding(max_steps, max_dims)
PE = pos_emb(np.zeros((1, max_steps, max_dims), np.float32))[0].numpy()

In [ ]:
i1, i2, crop_i = 100, 101, 150
p1, p2, p3 = 22, 60, 35
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(9, 5))
ax1.plot([p1, p1], [-1, 1], "k--", label="$p = {}$".format(p1))
ax1.plot([p2, p2], [-1, 1], "k--", label="$p = {}$".format(p2), alpha=0.5)
ax1.plot(p3, PE[p3, i1], "bx", label="$p = {}$".format(p3))
ax1.plot(PE[:,i1], "b-", label="$i = {}$".format(i1))
ax1.plot(PE[:,i2], "r-", label="$i = {}$".format(i2))
ax1.plot([p1, p2], [PE[p1, i1], PE[p2, i1]], "bo")
ax1.plot([p1, p2], [PE[p1, i2], PE[p2, i2]], "ro")
ax1.legend(loc="center right", fontsize=14, framealpha=0.95)
ax1.set_ylabel("$P_{(p,i)}$", rotation=0, fontsize=16)
ax1.grid(True, alpha=0.3)
ax1.hlines(0, 0, max_steps - 1, color="k", linewidth=1, alpha=0.3)
ax1.axis([0, max_steps - 1, -1, 1])
ax2.imshow(PE.T[:crop_i], cmap="gray", interpolation="bilinear", aspect="auto")
ax2.hlines(i1, 0, max_steps - 1, color="b")
cheat = 2 # need to raise the red line a bit, or else it hides the blue one
ax2.hlines(i2+cheat, 0, max_steps - 1, color="r")
ax2.plot([p1, p1], [0, crop_i], "k--")
ax2.plot([p2, p2], [0, crop_i], "k--", alpha=0.5)
ax2.plot([p1, p2], [i2+cheat, i2+cheat], "ro")
ax2.plot([p1, p2], [i1, i1], "bo")
ax2.axis([0, max_steps - 1, 0, crop_i])
ax2.set_xlabel("$p$", fontsize=16)
ax2.set_ylabel("$i$", rotation=0, fontsize=16)
save_fig("positional_embedding_plot")
plt.show()

In [ ]:
embed_size = 512; max_steps = 500; vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

##### **멀티-헤드 어텐션**

다음은 (매우) 간소화한 Transformer입니다(실제 구조는 스킵 연결, 층 정규화, 밀집 층 그리고 가장 중요하게 일반적인 어텐션이 아니라 멀티-헤드 어텐션을 가집니다):

In [ ]:
Z = encoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True)([Z, Z])

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z])
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(
    keras.layers.Dense(vocab_size, activation="softmax"))(Z)

다음은 기본적인 `MultiHeadAttention` 층의 구현입니다. 가까운 시일 내에 `keras.layers`에 추가될 것 같습니다. `kernel_size=1`인 (그리고 기본값 `padding="valid"`, `strides=1`을 사용하는) `Conv1D` 층은 `TimeDistributed(Dense(...))`과 같습니다.

In [ ]:
K = keras.backend

class MultiHeadAttention(keras.layers.Layer):
    def __init__(self, n_heads, causal=False, use_scale=False, **kwargs):
        self.n_heads = n_heads
        self.causal = causal
        self.use_scale = use_scale
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.dims = batch_input_shape[0][-1]
        self.q_dims, self.v_dims, self.k_dims = [self.dims // self.n_heads] * 3 # could be hyperparameters instead
        self.q_linear = keras.layers.Conv1D(self.n_heads * self.q_dims, kernel_size=1, use_bias=False)
        self.v_linear = keras.layers.Conv1D(self.n_heads * self.v_dims, kernel_size=1, use_bias=False)
        self.k_linear = keras.layers.Conv1D(self.n_heads * self.k_dims, kernel_size=1, use_bias=False)
        self.attention = keras.layers.Attention(causal=self.causal, use_scale=self.use_scale)
        self.out_linear = keras.layers.Conv1D(self.dims, kernel_size=1, use_bias=False)
        super().build(batch_input_shape)
    def _multi_head_linear(self, inputs, linear):
        shape = K.concatenate([K.shape(inputs)[:-1], [self.n_heads, -1]])
        projected = K.reshape(linear(inputs), shape)
        perm = K.permute_dimensions(projected, [0, 2, 1, 3])
        return K.reshape(perm, [shape[0] * self.n_heads, shape[1], -1])
    def call(self, inputs):
        q = inputs[0]
        v = inputs[1]
        k = inputs[2] if len(inputs) > 2 else v
        shape = K.shape(q)
        q_proj = self._multi_head_linear(q, self.q_linear)
        v_proj = self._multi_head_linear(v, self.v_linear)
        k_proj = self._multi_head_linear(k, self.k_linear)
        multi_attended = self.attention([q_proj, v_proj, k_proj])
        shape_attended = K.shape(multi_attended)
        reshaped_attended = K.reshape(multi_attended, [shape[0], self.n_heads, shape_attended[1], shape_attended[2]])
        perm = K.permute_dimensions(reshaped_attended, [0, 2, 1, 3])
        concat = K.reshape(perm, [shape[0], shape_attended[1], -1])
        return self.out_linear(concat)

In [ ]:
Q = np.random.rand(2, 50, 512)
V = np.random.rand(2, 80, 512)
multi_attn = MultiHeadAttention(8)
multi_attn([Q, V]).shape

### 언어 모델 분야의 최근 혁신

* 마스크드 언어 모델 (MLM)

* 다음 문장 예측 (NSP)